In [8]:
from research_assist.gsuite.drive.GoogleDriveHelper import GoogleDriveHelper

master_folder_name = "main"
drive_helper = GoogleDriveHelper(f"{master_folder_name}")
project_folder_id = drive_helper.create_new_folder("carbon_credit_mexico")
version_folder_id = drive_helper.create_new_folder(
  "v1", 
  parent_folder_id=project_folder_id
)
final_report_id = drive_helper.create_basic_document(
    "final report", parent_folder_id=version_folder_id
)
document_metadata = {
            "name": "carbon_credit",
            "mimeType": "application/vnd.google-apps.document",
            "parents": [version_folder_id],
}
# make the document
doc = (
  drive_helper.drive_service.files().create(body=document_metadata, fields="id").execute()
)
doc_id = doc.get("id")

In [2]:
from research_assist.researcher.Agent import ResearchAgent, load_secrets
from langchain_openai import ChatOpenAI
from tavily import TavilyClient

secrets = load_secrets()
model = ChatOpenAI(
    model="gpt-4o-mini", temperature=0, api_key=secrets["OPENAI_API_KEY"]
)
tavily = TavilyClient(api_key=secrets["TAVILY_API_KEY"])

agent = ResearchAgent(model, tavily)

Agent: 2025-02-28T23:05:31Z | INFO | Agent.py:70 | 27892 >>> Setting up agent graph


In [3]:
task = """
I need a research on Decentralized Science. What problems soe it solves, whaat the solution and what a re the main inicitives and where to improve.
"""
result = agent.run_task(task_description=task,max_revisions=3)

Agent: 2025-02-28T23:07:27Z | INFO | Agent.py:116 | 27892 >>> ####################
Agent: 2025-02-28T23:07:27Z | INFO | Agent.py:120 | 27892 >>> Node : initial_plan
Agent: 2025-02-28T23:07:27Z | INFO | Agent.py:122 | 27892 >>> Task : plan
Agent: 2025-02-28T23:07:27Z | INFO | Agent.py:124 | 27892 >>> # Research Report Outline: Decentralized Science

## Executive Summary
- **Purpose of the Report**: Briefly introduce the concept of Decentralized Science (DeSci) and its significance in the current scientific landscape.
- **Key Findings**: Summarize the main problems DeSci addresses, the solutions it offers, and the initiatives currently in place.
- **Recommendations for Improvement**: Highlight areas where DeSci can evolve and enhance its impact.

## 1. Introduction
- **Definition of Decentralized Science**: Explain what DeSci is, including its principles and how it differs from traditional scientific practices.
- **Importance of the Topic**: Discuss the relevance of DeSci in the context 

In [4]:
memories = agent.in_memory_store.search(("1", "memories"))

In [5]:
final_essay = agent.in_memory_store.search(("1", "memories"))[-3].dict()["value"][
    "memory"
]["write"]["draft"]

In [9]:
from research_assist.gsuite.docs.GoogleDocsHelper import GoogleDocsHelper

docs_helper = GoogleDocsHelper()

# add the document title 
title_end_index = docs_helper.create_doc_template_header(
    "carbon_credit", doc_id
)

# add the text
doc_end_index = docs_helper.write_text_to_doc(
    start_index=title_end_index, text=final_essay, doc_id=doc_id
)

In [13]:
from collections import defaultdict 

memories = agent.in_memory_store.search(("1", "memories"))

# this is needed because we may call some nodes several times 
# and we want to keep track of this so that we can make new documents
# for each call
seen_keys = set()
iterations = defaultdict(int)

# folder id where we want to write the documents
folder_id = f"{version_folder_id}"

for m in memories:
    data = m.dict()["value"]["memory"]
    available_keys = data.keys()
    node_key = list(available_keys)[0]
    unique_node_key = node_key + "_00"
    if unique_node_key in seen_keys:
        iterations[node_key] += 1
        unique_node_key = unique_node_key.replace("_00", "") + "_{:02d}".format(
            iterations[node_key]
        )

    print("-" * 20)
    print("Creating doc {}".format(unique_node_key))

    # get the text
    text = data[node_key][list(data[node_key].keys())[0]]

    # the tavily research output is a list, so convert it to a string
    if isinstance(text, list):
        text = "nn".join(text)

    # if anything else is not a string (e.g. the output of the accept node)
    # convert it to a string
    if not isinstance(text, str):
        text = str(text)

    # create document
    report_id = drive_helper.create_basic_document(
        unique_node_key, parent_folder_id=folder_id
    )

    # create header
    end_index = docs_helper.create_doc_template_header(unique_node_key, report_id)

    # fill document
    end_index = docs_helper.write_text_to_doc(
        start_index=end_index, text=text, doc_id=report_id
    )

    seen_keys.add(unique_node_key)

--------------------
Creating doc initial_plan_00
--------------------
Creating doc do_research_00
--------------------
Creating doc write_00
--------------------
Creating doc editor_00
--------------------
Creating doc review_00
--------------------
Creating doc research_revise_00
--------------------
Creating doc write_01
--------------------
Creating doc editor_01
--------------------
Creating doc accept_00
